# Duplicate header transformation challenge

In [7]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps, describe, Schema, validate, Dialect, table
from tabulate import tabulate
import re
from collections import Counter
import os
import csv

# https://framework.frictionlessdata.io/docs/guides/validating-data.html#validation-report


In [8]:
schema = Schema.describe('Purdue_ACRE_DTC_2021_1_1.csv')
schema.to_yaml("harvest.schema.yaml")

df='Purdue_ACRE_DTC_2021_1_1.csv'

### Using python's csv library
I wrote this custom code to identify a duplicate field using python csv library. The end goal is to transform the files with duplicate fields.

In [9]:
duplicate_col_files=[]
def check_and_update_duplicate_columns(file_path, destination_folder):
    with open(file_path, mode='r', newline='', encoding='mbcs') as file:
        reader = csv.reader(file)
        headers = next(reader)
        data = list(reader)

    # Check for duplicate columns and update names
    seen = {}
    new_headers = []
    for col in headers:
        if col in seen:
            seen[col] += 1
            new_headers.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_headers.append(col)

    duplicate_columns = [col for col, count in seen.items() if count > 0]

    if duplicate_columns:
        print("Duplicate columns found:", duplicate_columns)
       


check_and_update_duplicate_columns(df, 'duplicate_col_folder')

Duplicate columns found: ['Diff Status']


### Duplicate header using Frictionless data transform
Jarod mentioned that he wants minimal custom code so we can document the pipeline based on Frictionless glossary. So I tried using Frictionless transform

In Frictionless, Duplicate fields are read as Field and Field2. The challenge is that I cannot perform any function (move, pack, merge etc) on the duplicate field (Field2) except to remove the field (https://framework.frictionlessdata.io/docs/steps/field.html#remove-field). This means I cannot directly work with Field2 and in someways, Frictionless does not 'identify' it. 

I used this method on a demo csv file and it worked but it fails when I use on the file from SMS 

In [10]:
source = Resource(path="harvest/all_harvest/Purdue_ACRE_DTC_2021_1_1.csv")
target = transform(
    source,
    steps=[

        steps.field_merge(name="name4", from_names=["Diff Status"], preserve=False), # Change Field to a temporary name (name4). This automatically changes Field2 to Field

        # This step is throwing errors (when you run target.to_view()). I tried it on a demo csv file and it worked
        steps.field_add(name="name3", formula='Diff Status*1'), # A new field (name3) is created from the duplicate (now identified as Field) so that Frictionless can identify
        
        steps.field_remove(names=["Diff Status2"]), # Get rid of the duplicate (now identified as Field)
        steps.field_merge(name="Diff Status", from_names=["name4"], preserve=False), # Rename temp field to original name
        steps.field_merge(name="Diff Status2", from_names=["name3"], preserve=False), # Rename temp field to original name
        steps.field_move(name="Diff Status", position=14), # Move Field back to original position
        steps.field_move(name="Diff Status2", position=20), # Move Field back to original position

    ]
)
print(target.schema)
print(target.to_view())

{'fields': [{'name': 'Longitude', 'type': 'number'},
            {'name': 'Latitude', 'type': 'number'},
            {'name': 'Field', 'type': 'string'},
            {'name': 'Dataset', 'type': 'string'},
            {'name': 'Product', 'type': 'integer'},
            {'name': 'Obj. Id', 'type': 'integer'},
            {'name': 'Track(deg)', 'type': 'number'},
            {'name': 'Swth Wdth(ft)', 'type': 'number'},
            {'name': 'Distance(ft)', 'type': 'number'},
            {'name': 'Duration(s)', 'type': 'number'},
            {'name': 'Elevation(ft)', 'type': 'number'},
            {'name': 'Area Count', 'type': 'string'},
            {'name': 'Speed(mph)', 'type': 'number'},
            {'name': 'Diff Status', 'type': 'string'},
            {'name': 'Time', 'type': 'string'},
            {'name': 'X Offset(ft)', 'type': 'number'},
            {'name': 'Y Offset(ft)', 'type': 'number'},
            {'name': 'Satellites', 'type': 'string'},
            {'name': 'Hding Veh(deg

FrictionlessException: [step-error] Step is not valid: "field_add" raises "invalid syntax (<unknown>, line 1)" 

In [ ]:
report = validate(target, schema=schema,limit_errors=2)
if(report.valid == False):
    pprint(report)

{'valid': False,
 'stats': {'tasks': 1, 'errors': 2, 'warnings': 1, 'seconds': 0.034},
 'warnings': [],
 'errors': [],
 'tasks': [{'name': 'purdue_acre_dtc_2021_1_1',
            'type': 'table',
            'valid': False,
            'place': '<memory>',
            'labels': ['-86.98998285',
                       '40.47252643',
                       'DTC',
                       'Harvest (GreenStar 3 Monitor)',
                       '1',
                       '100',
                       '269.50',
                       '20.00',
                       '5.525',
                       '1.000',
                       '719.76',
                       'On',
                       '3.767',
                       '10/19/2021',
                       '0.00',
                       '0.00',
                       'N/A',
                       '0.00',
                       'N/A',
                       '0.00',
                       '0.00',
                       '0.00',
                